In [8]:
import glob
import pandas as pd
df = pd.DataFrame(columns=["content", "sentiment"])
for fn in glob.glob("aclImdb/train/pos/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    content = f.read
    f.close()
    s = pd.Series([content, 1], index=["content", "sentiment"])
    df = df.append(s, ignore_index=True)
for fn in glob.glob("aclImdb/train/neg/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    content = f.read
    f.close()
    s = pd.Series([content, 1], index=["content", "sentiment"])
    df = df.append(s, ignore_index=True)
df

,content,sentiment
0,<built-in method read of _io.TextIOWrapper obj...,1
1,<built-in method read of _io.TextIOWrapper obj...,1
2,<built-in method read of _io.TextIOWrapper obj...,1
3,<built-in method read of _io.TextIOWrapper obj...,1
4,<built-in method read of _io.TextIOWrapper obj...,1
...,...,...
24995,<built-in method read of _io.TextIOWrapper obj...,1
24996,<built-in method read of _io.TextIOWrapper obj...,1
24997,<built-in method read of _io.TextIOWrapper obj...,1
24998,<built-in method read of _io.TextIOWrapper obj...,1


In [9]:
testdf = pd.DataFrame(columns=["content", "sentiment"])
for fn in glob.glob("aclImdb/test/pos/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    content = f.read
    f.close()
    s = pd.Series([content, 1], index=["content", "sentiment"])
    testdf = testdf.append(s, ignore_index=True)
for fn in glob.glob("aclImdb/test/neg/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    content = f.read
    f.close()
    s = pd.Series([content, 1], index=["content", "sentiment"])
    testdf = testdf.append(s, ignore_index=True)
testdf

,content,sentiment
0,<built-in method read of _io.TextIOWrapper obj...,1
1,<built-in method read of _io.TextIOWrapper obj...,1
2,<built-in method read of _io.TextIOWrapper obj...,1
3,<built-in method read of _io.TextIOWrapper obj...,1
4,<built-in method read of _io.TextIOWrapper obj...,1
...,...,...
24995,<built-in method read of _io.TextIOWrapper obj...,1
24996,<built-in method read of _io.TextIOWrapper obj...,1
24997,<built-in method read of _io.TextIOWrapper obj...,1
24998,<built-in method read of _io.TextIOWrapper obj...,1


In [12]:
from keras.preprocessing.text import Tokenizer
# 先創造擬 care 的詞的辭典, 1000業的辭典(等一下每個詞都是1000維度的向量)
tok = Tokenizer(num_words=1000)
tok.fit_on_texts(df["content"])

AttributeError: 'builtin_function_or_method' object has no attribute 'lower'

In [ ]:
# 倒底1000個詞是哪一個1000的詞(抓出出現次數最多的)
tok.word_index

In [ ]:
# 把每篇文章的每個詞都轉化為數字(1000維度的向量)
train_seq = tok.texts_to_sequences(df["content"])
test_seq = tok.texts_to_sequences(testdf["content"])
# 因為每篇 文章不一樣多, 出現最多的文章1496個詞, 其他文章就會補上 NaN
pd.DataFrame(train_seq)

In [ ]:
from ketas.processing.sequences import pad_sequences
train_pad = pad_sequences(train_seq, maxlen=100)
test_pad = pad_sequences(test_seq, maxlen=100)
# 如郭看到 0 , 補短出現的結果
pd.DataFrame(train_pad)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flattten, Embedding, Dropout
model = Sequential()
model.add(Embedding(input_dim=1000, out_dim=32, input_length=100))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(1, activation="sigmold"))
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
model.fit(train_pad, df["sentiment"], batch_size=200, epochs=3,
          verbos=2, validation_split=0.1)

In [ ]:
model.evaluate(test_pad, testdf["sentiment"])

In [ ]:
# 儲存模型
model.save("sentiment.h5")

In [ ]:
# 讀取模型(如果你要繼續訓練, 不用繼續compile)
from keras.models import loas_model
l = load_model("sentiment.h5")
l.evaluate(test_pad, testdf["sentiment"])

In [ ]:
# 你可選擇中間層當作輸出
# 先 model.layers 找出你要的那層 model.layers[2].out (=> tf.tensor)
# 然後 model.input
from keras.models import Model
partial = Model(model.input, model.layers[0].output)
# 先取出第一篇文章 : 100個詞 (32維度的向量)
partial.predict(test_pad)[0].shape